# Apache Kafka Event Producer 1

Submitted by : Mayank Bhardwaj, Neha Jain, Rishupal Singh Chabbra

In the below code, we have created an Event Producer using Kafka Producer. This will act as Producer 1 for the streaming data and will randomly produce the data for climate_streaming.csv at an interval of 5 seconds each. Additionally, sender_id = "P1" and created_time = current timestamp is appended to the record so as to identify the producer at the consumer end.

#### Topic "climate" is used for the Kafka stream

In [1]:
import sys
import time
import random
import datetime
import csv

climate_streaming = []
with open('climate_streaming.csv') as csv_file:
    csv_reader = csv.reader(csv_file,delimiter=',')
    headers = next(csv_reader)    #skipping header
    for row in csv_reader:
        # loading each row as json record and appending to a list so as to send to the consumer
        record = {'latitude': row[0]  , 
                  'longitude': row[1] , 
                  'air_temperature_celcius': row[2] , 
                  'relative_humidity': row[3] , 
                  'windspeed_knots': row[4] , 
                  'max_wind_speed': row[5] , 
                  'precipitation' : row[6],
                  'sender_id':'P1'            
        }
        
        climate_streaming.append(record)

In [2]:
# import statements

from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt
        
        
def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')  #converting the message to bytes to send
        producer_instance.send(topic_name, key=key_bytes, value=data) #sending the message
        producer_instance.flush() # flushing the producer
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex: # exception in case of any issues encounters
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        # connecting to the Kafka server and creating the KafkaProducer object
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'], 
                                  value_serializer=lambda x:dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
   
    topic = 'climate' #Topic for Kafka is climate
    
    print('Publishing records..')
    while True:
        producer = connect_kafka_producer() # connecting to the Kafka and using its object to send messages
        try:
            data = climate_streaming[random.randrange(len(climate_streaming)-1)] # random feeding of the messages
            # appending the created_time as current_timestamp
            data['created_time']=str(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')) 
            # publishing the message as json, converted to bytes before sending
            publish_message(producer, topic, 'jsondata', data) 
            sleep(5)  # creating an interval of 5 seconds to send the data
        finally:
            producer.close()

Publishing records..
Message published successfully. Data: {'sender_id': 'P1', 'precipitation': ' 0.00I', 'max_wind_speed': '13', 'windspeed_knots': '9.3', 'created_time': '2019-05-28 01:05:59', 'latitude': '-37.611', 'longitude': '149.277', 'air_temperature_celcius': '18', 'relative_humidity': '49.4'}
Message published successfully. Data: {'sender_id': 'P1', 'precipitation': ' 0.00I', 'max_wind_speed': '14', 'windspeed_knots': '7.9', 'created_time': '2019-05-28 01:06:04', 'latitude': '-37.858', 'longitude': '143.428', 'air_temperature_celcius': '13', 'relative_humidity': '50.4'}
Message published successfully. Data: {'sender_id': 'P1', 'precipitation': ' 0.00G', 'max_wind_speed': '20', 'windspeed_knots': '14.4', 'created_time': '2019-05-28 01:06:09', 'latitude': '-37.5268', 'longitude': '143.0551', 'air_temperature_celcius': '8', 'relative_humidity': '35'}
Message published successfully. Data: {'sender_id': 'P1', 'precipitation': ' 0.00I', 'max_wind_speed': '14', 'windspeed_knots': '7

KeyboardInterrupt: 